In [32]:
import sys
import importlib as imp
if ('Jupytils' in sys.modules):
    reloaded = imp.reload(Jupytils)
else:
    import Jupytils
    

## Naive Bayes

### References:
* http://www.analyticsvidhya.com/blog/2015/09/naive-bayes-explained/

#### Some Theory
Bayes Theorem

Gaussian probability function is defined as:
$$ P(x) = \frac{1}{\sigma \sqrt{2\pi }} e^{- \frac{{(x-\mu)}^2}{2 \sigma^2} }$$

Also we know from Bayes theorem:

$$ P(X|Y) = \frac{P( X \cap Y)}{P(Y)} = \frac{P(Y|X) P(X)}{\sum_x{P(Y|x)} } $$



In [17]:
fileName="../data/pima-indians-diabetes.csv"

df  = LoadDataSet(fileName);
displayDFs([df], maxrows=12)


count,768.000,768.000,768.000,768.000,768.000,768.000,768.000,768.000,768.000
mean,3.845,120.895,69.105,20.536,79.799,31.993,0.472,33.241,0.349
std,3.370,31.973,19.356,15.952,115.244,7.884,0.331,11.760,0.477
min,0.000,0.000,0.000,0.000,0.000,0.000,0.078,21.000,0.000
25%,1.000,99.000,62.000,0.000,0.000,27.300,0.244,24.000,0.000
50%,3.000,117.000,72.000,23.000,30.500,32.000,0.372,29.000,0.000
75%,6.000,140.250,80.000,32.000,127.250,36.600,0.626,41.000,1.000
max,17.000,199.000,122.000,99.000,846.000,67.100,2.420,81.000,1.000
,,,,,,,,,
,Num_Pregnant (int64),Glucose_c (int64),BP (int64),SkinThick (int64),Serum_Insulin (int64),BMI (float64),Diabetes (float64),Age (int64),Diabetic (int64)
0,6,148,72,35,0,33.600,0.627,50,1


In [18]:
# Example of Naive Bayes implemented from Scratch in Python
import csv
import random
import math

def loadCsv(filename):
    lines = csv.reader(open(filename, "rt"))
    dataset = list(lines)
    dataset = dataset[3:]
    for i in range(len(dataset)):
        dataset[i] = [float(x) for x in dataset[i]]
    return dataset

def splitDataset(dataset, splitRatio):
	trainSize = int(len(dataset) * splitRatio)
	trainSet = []
	testSet = list(dataset)
	while len(trainSet) < trainSize:
		index = random.randrange(len(testSet))
		trainSet.append(testSet.pop(index))
	return [trainSet, testSet]

def separateByClass(dataset):
	separated = {}
	for i in range(len(dataset)):
		vector = dataset[i]
		if (vector[-1] not in separated):
			separated[vector[-1]] = []
		separated[vector[-1]].append(vector)
	return separated

def mean(numbers):
	return sum(numbers)/float(len(numbers))

def stdev(numbers):
	avg = mean(numbers)
	variance = sum([pow(x-avg,2) for x in numbers])/float(len(numbers)-1)
	return math.sqrt(variance)

def summarize(dataset):
	#summaries = [(mean(attribute), stdev(attribute)) for attribute in zip(*dataset)]
	summaries = [(np.array(a).mean(), np.array(a).std(), np.array(a).var()) for a in zip(*dataset)]
	del summaries[-1]
	return summaries

def summarizeByClass(dataset):
    separated = separateByClass(dataset)
    summaries = {}
    labels={}
    y = list(zip(*dataset))[-1]
    for classValue, instances in separated.items():
        summaries[classValue] = summarize(instances)
        labels[classValue] = y.count(classValue)/len(y)
    return summaries, labels

def calculateProbability(x, mean, stdev, var=None):
    if (var is None):
        var = math.pow(stdev,2)
    exponent = math.exp(-(math.pow(x-mean,2)/(2*var)))
    return (1 / (math.sqrt(2*math.pi) * stdev)) * exponent

def calculateClassProbabilities(summaries, labels, inputVector):
	probabilities = {}
	for classValue, classSummaries in summaries.items():
		probabilities[classValue] = labels[classValue]
		for i in range(len(classSummaries)):
			mean, stdev, var = classSummaries[i]
			x = inputVector[i]
			probabilities[classValue] *= calculateProbability(x, mean, stdev, var)
	return probabilities
			
def predict(summaries, labels, inputVector):
	probabilities = calculateClassProbabilities(summaries, labels, inputVector)
	bestLabel, bestProb = None, -1
	for classValue, probability in probabilities.items():
		if bestLabel is None or probability > bestProb:
			bestProb = probability
			bestLabel = classValue
	return bestLabel

def getPredictions(summaries, labels, testSet):
	predictions = []
	for i in range(len(testSet)):
		result = predict(summaries, labels, testSet[i])
		predictions.append(result)
	return predictions

def getProbs(summaries, labels, testSet):
    r = []
    for i in range(len(testSet)):
        p = calculateClassProbabilities(summaries, labels, testSet[i])
        r.append(p)
    return r


def getAccuracy(testSet, predictions):
	correct = 0
	for i in range(len(testSet)):
		if testSet[i][-1] == predictions[i]:
			correct += 1
	return (correct/float(len(testSet))) * 100.0

## - Use of locally implemented Naive-Bayes
## 
filename = '../data/pima-indians-diabetes.csv'
splitRatio = 0.67
dataset = loadCsv(filename)
trainingSet, testSet = splitDataset(dataset, splitRatio)
print('Split {0} rows into train={1} and test={2} rows'.format(len(dataset), len(trainingSet), len(testSet)) )
# prepare model
summaries, labels = summarizeByClass(trainingSet)
# test model

#testSet = testSet[0:50]
predictions = getPredictions(summaries, labels, testSet)

probs = getProbs(summaries,labels, testSet)
accuracy = getAccuracy(testSet, predictions)
print('Accuracy: {0}%'.format(accuracy))

Split 768 rows into train=514 and test=254 rows
Accuracy: 75.59055118110236%


### Using Python tools

In [19]:
def SplitXy(Xy):
    Xy10=Xy[0:8]
    Xy10 = Xy;
    #print Xy10
    #print "========"
    zXy10=list(zip(*Xy10))
    y= zXy10[-1]
    del zXy10[-1]
    z1=zip(*zXy10)
    X=[list(t) for t in z1]
    return np.array(X), np.array(y)

from sklearn.naive_bayes import GaussianNB
X,y = SplitXy(trainingSet) 
Xt,yt = SplitXy(testSet)

model = GaussianNB()
model.fit(X, y)


GaussianNB()

In [20]:
### Compare the models built by Python

print ("Class: 0")
for i,j in enumerate(model.theta_[0]):
    print ("({:8.2f} {:9.2f} {:9.2f} )".format(j, sqrt(model.sigma_[0][i]), model.sigma_[0][i]) , end="")
    print ("==> ", summaries[0][i])
    
print ("Class: 1")
for i,j in enumerate(model.theta_[1]):
    print ("({:8.2f} {:9.2f} {:9.2f} )".format(j, sqrt(model.sigma_[1][i]), model.sigma_[1][i]) , end="")
    print ("==> ", summaries[1][i])

Class: 0
(    3.34      3.08      9.50 )==>  (3.3362573099415203, 3.0818893770902944, 9.4980421326220039)
(  110.29     25.27    638.34 )==>  (110.28654970760233, 25.265486139195268, 638.34478984986822)
(   67.88     18.66    348.26 )==>  (67.883040935672511, 18.661757001376941, 348.26117437844124)
(   19.61     14.97    224.00 )==>  (19.608187134502923, 14.966580419790512, 223.99852946205672)
(   67.13     99.98   9995.07 )==>  (67.131578947368425, 99.975334369544612, 9995.0674823022473)
(   30.07      7.73     59.72 )==>  (30.070467836257311, 7.7277868277284574, 59.718689254813448)
(    0.43      0.31      0.10 )==>  (0.42692982456140349, 0.31358089471705414, 0.098332977531548182)
(   31.53     12.20    148.93 )==>  (31.529239766081872, 12.203585850444897, 148.92750760917889)
Class: 1
(    4.75      3.87     14.99 )==>  (4.75, 3.8716696116010918, 14.989825581395349)
(  142.34     32.43   1051.81 )==>  (142.34302325581396, 32.431570323538487, 1051.8067536506221)
(   71.08     21.59   

In [21]:
Pt = model.predict_proba(Xt)
Pt[0:10]

array([[ 0.98,  0.02],
       [ 0.  ,  1.  ],
       [ 0.95,  0.05],
       [ 0.98,  0.02],
       [ 0.44,  0.56],
       [ 0.9 ,  0.1 ],
       [ 0.44,  0.56],
       [ 0.59,  0.41],
       [ 0.22,  0.78],
       [ 0.99,  0.01]])

In [22]:
p1 = model.predict(Xt)
for i in range(len(predictions)):
    if (not predictions[i] == p1[i] ):
        print ("{} differ {} {}, {} ".format(i, predictions[i] , p1[i] , Xt[i]), end="\n")
        break;

In [23]:
probs[0:10]

[{0.0: 1.7269584007759185e-12, 1.0: 3.5371698227370762e-14},
 {0.0: 1.3195013352802532e-21, 1.0: 9.0712040498185301e-19},
 {0.0: 1.1614238966239661e-12, 1.0: 6.1916301736486025e-14},
 {0.0: 9.7455685872541638e-13, 1.0: 2.4207527820839252e-14},
 {0.0: 1.4362899842131421e-16, 1.0: 1.8233351782754195e-16},
 {0.0: 2.9260237827604278e-12, 1.0: 3.3625739366325391e-13},
 {0.0: 8.2835298503228159e-14, 1.0: 1.0505979945330254e-13},
 {0.0: 2.4986320909581826e-13, 1.0: 1.7343153838510679e-13},
 {0.0: 4.2898801396326906e-14, 1.0: 1.5162828048607136e-13},
 {0.0: 1.6757952169521505e-12, 1.0: 1.6961759261352896e-14}]

In [24]:
X,y = SplitXy(trainingSet)
yy=np.array(y)
y0=yy[yy==0]
y1=yy[yy==1]
p0=len(y0)/len(y)
p1=len(y1)/len(y)
print(len(y0), len(y1), len(y), p0, p1)

342 172 514 0.6653696498054474 0.3346303501945525


In [31]:
from dill.source import getsource
print(getsource(model.predict_log_proba))
print(getsource(model.predict_proba))

    def predict_log_proba(self, X):
        """
        Return log-probability estimates for the test vector X.

        Parameters
        ----------
        X : array-like, shape = [n_samples, n_features]

        Returns
        -------
        C : array-like, shape = [n_samples, n_classes]
            Returns the log-probability of the samples for each class in
            the model. The columns correspond to the classes in sorted
            order, as they appear in the attribute `classes_`.
        """
        jll = self._joint_log_likelihood(X)
        # normalize by P(x) = P(f_1, ..., f_n)
        log_prob_x = logsumexp(jll, axis=1)
        return jll - np.atleast_2d(log_prob_x).T

    def predict_proba(self, X):
        """
        Return probability estimates for the test vector X.

        Parameters
        ----------
        X : array-like, shape = [n_samples, n_features]

        Returns
        -------
        C : array-like, shape = [n_samples, n_classes]
            Re

In [26]:
print(getsource(model._joint_log_likelihood))

    def _joint_log_likelihood(self, X):
        check_is_fitted(self, "classes_")

        X = check_array(X)
        joint_log_likelihood = []
        for i in range(np.size(self.classes_)):
            jointi = np.log(self.class_prior_[i])
            n_ij = - 0.5 * np.sum(np.log(2. * np.pi * self.sigma_[i, :]))
            n_ij -= 0.5 * np.sum(((X - self.theta_[i, :]) ** 2) /
                                 (self.sigma_[i, :]), 1)
            joint_log_likelihood.append(jointi + n_ij)

        joint_log_likelihood = np.array(joint_log_likelihood).T
        return joint_log_likelihood



In [28]:
print(model.class_prior_[0], model.class_prior_[1])

0.665369649805 0.334630350195


In [30]:
jll = model._joint_log_likelihood(Xt)
jll[0:20]

array([[-27.08, -30.97],
       [-48.07, -41.54],
       [-27.48, -30.41],
       [-27.66, -31.35],
       [-36.48, -36.24],
       [-26.56, -28.72],
       [-30.12, -29.88],
       [-29.02, -29.38],
       [-30.78, -29.52],
       [-27.11, -31.71],
       [-29.53, -30.04],
       [-28.15, -30.62],
       [-29.99, -29.33],
       [-27.86, -31.98],
       [-36.61, -31.82],
       [-33.56, -30.13],
       [-34.52, -33.81],
       [-28.77, -29.38],
       [-30.87, -34.23],
       [-28.11, -29.94]])

In [82]:
#Import Library of Gaussian Naive Bayes model

#assigning predictor and target variables

#Output: ([3,4])


[3 4]


In [99]:
x.mean(axis=0)
x.var(axis=0)

array([ 4.58,  7.  ])

In [92]:
x1 = x[y==3]


In [154]:
from sklearn.naive_bayes import GaussianNB
import numpy as np

class myNB:
    def __init__(self):
        pass;
    
    def fit(self,x,y):
        labels = unique(y)
        params = defaultdict(dict)
        for k in (labels):
            x1 = x[y==k]
            params[k]['class_prob'] = len(x1)/len(x);
            params[k]['mean'] = x1.mean(axis=0);
            params[k]['var' ] = x1.var(axis=0);
            
        self.labels = labels
        self.params = params
    
    def Gaussian(self, x, mean, var):
        e = math.exp(-(math.pow(x-mean,2)/(2*var)))
        return (1 / (math.sqrt(2*math.pi * var)) * e)

    def predict_proba1(self, x):
        p = {}
        for k, parms in self.params.items():
            p[k] = parms['class_prob']
            for i in range(len(parms['mean'])):
                mean, var = parms['mean'][i], parms['var'][i]
                p[k] *= self.Gaussian( x[i], mean, var)
        return p
    
    def predict_proba(self, Xt, dump=False):
        r = []
        for x in Xt:
            r.append(self.predict_proba1(x))
        
        if (dump):
            for i in r:
                p = max(i, key=i.get)
                print(" Predicted ==>", p, " Labels: ", self.labels, " Probs: " ,i)
        return r;
            
    def predict(self, Xt, dump=False):
        pass
    
    def dump(self):
        print("My NB : labels =>", self.labels);
        for k in self.labels:
            print(k, self.params[k])
            #print("{} ==> mean: {:8.2f}  Var: {:9.2f} )".format(j, self.params[k]) , end="")
        pass

X = np.array([[-3,7],[1,5], [1,2], [-2,0], [2,3], [-4,0], [-1,1], [1,1], [-2,2], [2,7], [-4,1], [-2,7]])
y = np.array([0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1])

myModel = myNB()

# Train the model using the training sets 
pyModel.fit(X, y)
                
myModel.fit(X,y)
myModel.dump()

predicted_prob= myModel.predict_proba([[1,2],[3,4]], True)

#pyModel = GaussianNB()
#print ("Gaussian NB parameters: ")
#for k in myModel.labels:
#    for i,j in enumerate(pyModel.theta_[k]):
        #print (k)
#        print ("{} ({:6.2f} {:6.2f} )".format(k, j, model.sigma_[k][i]) )
    
#Predict Output 
#predicted= model.predict([[1,2],[3,4]])
#print( predicted)

My NB : labels => [0 1]
0 {'var': array([ 3.1 ,  5.96]), 'mean': array([-1.43,  2.43]), 'class_prob': 0.5833333333333334}
1 {'var': array([ 5.76,  7.36]), 'mean': array([-0.2,  3.8]), 'class_prob': 0.4166666666666667}
 Predicted ==> 0  Labels:  [0 1]  Probs:  {0: 0.008217910623911775, 1: 0.007212402228071897}
 Predicted ==> 1  Labels:  [0 1]  Probs:  {0: 0.0007437940158082369, 1: 0.004175793880737992}


In [110]:
pyModel.theta_

array([[-1.43,  2.43],
       [-0.2 ,  3.8 ]])

In [148]:
r = {}
r['a'] = 90
r['b'] = 8
max(r, key=r.get)


'a'